In [1]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from collections import OrderedDict
from utils import read_data, normalize
from model import get_KMeans
# from model import PCA

In [2]:
used_countries = ['USA', 'CHN', 'JPN', 'DEU', 'IND', 
                  'GBR', 'FIN', 'BRA', 'ITA', 'CAN', 
                  'RUS', 'ZAF', 'AUS', 'ESP', 'MEX',
                  'IDN', 'TUR', 'NLD', 'SAU', 'CHE']
country_covid_data = read_data('data.csv', used_countries)

In [3]:
normalized_data = OrderedDict()
for country in used_countries:
    tem_data = np.array(country_covid_data[country]['Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'])
    normalized_data[country] = normalize(tem_data)

In [4]:
from sklearn.cluster import KMeans
km = get_KMeans('sklearn', 4)  #将数据集分为2类
Y = np.array([normalized_data[idx] for idx in normalized_data])
y_pre = km.fit_predict(Y)


In [11]:
y_pre
for i in range(4):
   for j in range(len(used_countries)):
      if j == i:
         print(used_countries[j])
   print('\n')

USA


CHN


JPN


DEU


